In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/27 14:15:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!hdfs dfs -cp sf.csv hdfs://nn:9000/sf.csv

In [3]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/sf.csv"))

In [4]:
from pyspark.sql.functions import col, expr
cols = [col(c).alias(c.replace(" ", "_")) for c in df.columns]
df.select(cols).write.format("parquet").save("hdfs://nn:9000/sf.parquet")

23/10/27 14:16:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
!hdfs dfs -rm hdfs://nn:9000/sf.csv

Deleted hdfs://nn:9000/sf.csv


In [11]:
df = spark.read.format("parquet").load("hdfs://nn:9000/sf.parquet")

In [13]:
df.limit(3).toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Call_Type_Group,Number_of_Alarms,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods
0,201340825,T06,20055834,Other,05/13/2020,05/13/2020,05/13/2020 08:51:30 AM,05/13/2020 08:51:30 AM,05/13/2020 08:54:56 AM,05/13/2020 08:55:10 AM,...,Alarm,1,TRUCK,1,2,8,Mission,201340825-T06,POINT (-122.42251458613262 37.770727334245194),20
1,201241707,FB3,20052084,Outside Fire,05/03/2020,05/03/2020,05/03/2020 01:46:02 PM,05/03/2020 01:46:02 PM,05/03/2020 01:46:11 PM,05/03/2020 01:47:41 PM,...,Alarm,1,SUPPORT,2,1,3,Financial District/South Beach,201241707-FB3,POINT (-122.3946511276673 37.79530574375478),8
2,201083222,AM118,20045948,Medical Incident,04/17/2020,04/17/2020,04/17/2020 09:19:43 PM,04/17/2020 09:19:43 PM,04/17/2020 09:21:10 PM,04/17/2020 09:21:55 PM,...,Non Life-threatening,1,PRIVATE,2,2,6,South of Market,201083222-AM118,POINT (-122.41071806030364 37.778959763358266),34


In [14]:
df.createTempView("calls")

In [38]:
df.withColumnRenamed("Neighborhooods_-_Analysis_Boundaries", "area").createOrReplaceTempView("calls")

In [20]:
spark.sql("SELECT * FROM calls LIMIT 3").toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Call_Type_Group,Number_of_Alarms,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods
0,201340825,T06,20055834,Other,05/13/2020,05/13/2020,05/13/2020 08:51:30 AM,05/13/2020 08:51:30 AM,05/13/2020 08:54:56 AM,05/13/2020 08:55:10 AM,...,Alarm,1,TRUCK,1,2,8,Mission,201340825-T06,POINT (-122.42251458613262 37.770727334245194),20
1,201241707,FB3,20052084,Outside Fire,05/03/2020,05/03/2020,05/03/2020 01:46:02 PM,05/03/2020 01:46:02 PM,05/03/2020 01:46:11 PM,05/03/2020 01:47:41 PM,...,Alarm,1,SUPPORT,2,1,3,Financial District/South Beach,201241707-FB3,POINT (-122.3946511276673 37.79530574375478),8
2,201083222,AM118,20045948,Medical Incident,04/17/2020,04/17/2020,04/17/2020 09:19:43 PM,04/17/2020 09:19:43 PM,04/17/2020 09:21:10 PM,04/17/2020 09:21:55 PM,...,Non Life-threatening,1,PRIVATE,2,2,6,South of Market,201083222-AM118,POINT (-122.41071806030364 37.778959763358266),34


In [22]:
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |    calls|       true|
+---------+---------+-----------+



In [23]:
spark.sql("""
SELECT *
FROM calls
WHERE Call_Type LIKE "Odor%"
LIMIT 10
""").toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Call_Type_Group,Number_of_Alarms,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods
0,201121216,E19,20047272,Odor (Strange / Unknown),04/21/2020,04/21/2020,04/21/2020 11:30:07 AM,04/21/2020 11:36:26 AM,04/21/2020 11:37:05 AM,04/21/2020 11:37:21 AM,...,Alarm,1,ENGINE,1,9,7,West of Twin Peaks,201121216-E19,POINT (-122.46804645644951 37.72571348318412),41
1,201363227,KM10,20056915,Odor (Strange / Unknown),05/15/2020,05/15/2020,05/15/2020 07:31:23 PM,05/15/2020 07:36:16 PM,05/15/2020 07:36:53 PM,05/15/2020 07:37:55 PM,...,Alarm,1,PRIVATE,4,7,1,Outer Richmond,201363227-KM10,POINT (-122.47938019994935 37.78130072448512),29
2,201301903,E31,20054576,Odor (Strange / Unknown),05/09/2020,05/09/2020,05/09/2020 03:27:52 PM,05/09/2020 03:30:54 PM,05/09/2020 03:31:15 PM,05/09/2020 03:32:40 PM,...,Alarm,1,ENGINE,1,7,1,Outer Richmond,201301903-E31,POINT (-122.47509052922184 37.78149633273287),29
3,201490091,54,20061869,Odor (Strange / Unknown),05/28/2020,05/27/2020,05/28/2020 01:06:30 AM,05/28/2020 01:06:30 AM,05/28/2020 01:07:33 AM,05/28/2020 01:08:07 AM,...,Alarm,1,MEDIC,1,3,6,Financial District/South Beach,201490091-54,POINT (-122.38902618685223 37.78581422562703),8
4,201453236,E11,20060459,Odor (Strange / Unknown),05/24/2020,05/24/2020,05/24/2020 09:10:36 PM,05/24/2020 09:13:42 PM,05/24/2020 09:14:03 PM,05/24/2020 09:14:52 PM,...,Alarm,1,ENGINE,1,6,8,Noe Valley,201453236-E11,POINT (-122.42538046669073 37.752629190046235),22
5,201671674,B06,20069363,Odor (Strange / Unknown),06/15/2020,06/15/2020,06/15/2020 01:44:41 PM,06/15/2020 01:46:16 PM,06/15/2020 01:48:14 PM,06/15/2020 01:48:14 PM,...,Alarm,1,CHIEF,3,6,9,Bernal Heights,201671674-B06,POINT (-122.42110563099509 37.74521073243275),2
6,201671674,T11,20069363,Odor (Strange / Unknown),06/15/2020,06/15/2020,06/15/2020 01:44:41 PM,06/15/2020 01:46:16 PM,06/15/2020 01:53:28 PM,06/15/2020 01:54:50 PM,...,Alarm,1,TRUCK,2,6,9,Bernal Heights,201671674-T11,POINT (-122.42110563099509 37.74521073243275),2
7,201650458,E37,20068546,Odor (Strange / Unknown),06/13/2020,06/12/2020,06/13/2020 05:41:55 AM,06/13/2020 05:43:34 AM,06/13/2020 05:43:46 AM,06/13/2020 05:45:46 AM,...,Alarm,1,ENGINE,1,10,10,Potrero Hill,201650458-E37,POINT (-122.40103723238265 37.753373613396604),26
8,201490091,T08,20061869,Odor (Strange / Unknown),05/28/2020,05/27/2020,05/28/2020 01:06:30 AM,05/28/2020 01:06:30 AM,05/28/2020 01:07:33 AM,05/28/2020 01:11:02 AM,...,Alarm,1,TRUCK,3,3,6,Financial District/South Beach,201490091-T08,POINT (-122.38902618685223 37.78581422562703),8
9,201592472,E15,20066343,Odor (Strange / Unknown),06/07/2020,06/07/2020,06/07/2020 08:11:43 PM,06/07/2020 08:13:49 PM,06/07/2020 08:13:57 PM,06/07/2020 08:15:41 PM,...,Alarm,1,ENGINE,1,9,11,Outer Mission,201592472-E15,POINT (-122.44100649677766 37.724985389851604),28


In [28]:
spark.sql("""
SELECT *
FROM calls
WHERE Call_Type LIKE "Odor%"
""").write.mode("ignore").saveAsTable("stinky")

In [29]:
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|   stinky|      false|
|         |    calls|       true|
+---------+---------+-----------+



In [30]:
!hdfs dfs -ls hdfs://nn:9000/user/hive/warehouse

Found 1 items
drwxr-xr-x   - root supergroup          0 2023-10-27 15:09 hdfs://nn:9000/user/hive/warehouse/stinky


In [31]:
!hdfs dfs -ls hdfs://nn:9000/user/hive/warehouse/stinky

Found 7 items
-rw-r--r--   3 root supergroup          0 2023-10-27 15:09 hdfs://nn:9000/user/hive/warehouse/stinky/_SUCCESS
-rw-r--r--   3 root supergroup     334214 2023-10-27 15:09 hdfs://nn:9000/user/hive/warehouse/stinky/part-00000-88ad621a-0876-4bf5-a5f2-6598a56ec936-c000.snappy.parquet
-rw-r--r--   3 root supergroup     370478 2023-10-27 15:09 hdfs://nn:9000/user/hive/warehouse/stinky/part-00001-88ad621a-0876-4bf5-a5f2-6598a56ec936-c000.snappy.parquet
-rw-r--r--   3 root supergroup     173436 2023-10-27 15:09 hdfs://nn:9000/user/hive/warehouse/stinky/part-00002-88ad621a-0876-4bf5-a5f2-6598a56ec936-c000.snappy.parquet
-rw-r--r--   3 root supergroup     209189 2023-10-27 15:09 hdfs://nn:9000/user/hive/warehouse/stinky/part-00003-88ad621a-0876-4bf5-a5f2-6598a56ec936-c000.snappy.parquet
-rw-r--r--   3 root supergroup     138716 2023-10-27 15:09 hdfs://nn:9000/user/hive/warehouse/stinky/part-00004-88ad621a-0876-4bf5-a5f2-6598a56ec936-c000.snappy.parquet
-rw-r--r--   3 root supergroup 

In [34]:
spark.sql("""
SELECT *
FROM calls
WHERE Call_Type LIKE "Odor%"
""").rdd.getNumPartitions()

6

In [35]:
spark.sql("""
SELECT *
FROM stinky
""").rdd.getNumPartitions()

2

In [36]:
spark.table("calls")  # pass a view to .table, get a DataFrame

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [37]:
spark.table("stinky")  # pass a table to .table, get a DataFrame

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

# Grouping, other Queries

In [39]:
spark.table("calls")

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, area: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [40]:
# what areas are in SF?

In [43]:
spark.sql("SELECT DISTINCT area FROM calls").collect()

[Row(area='Inner Sunset'),
 Row(area='Haight Ashbury'),
 Row(area='Lincoln Park'),
 Row(area='Japantown'),
 Row(area='None'),
 Row(area='North Beach'),
 Row(area='Lone Mountain/USF'),
 Row(area='Western Addition'),
 Row(area='Bernal Heights'),
 Row(area='Mission Bay'),
 Row(area='Hayes Valley'),
 Row(area='Financial District/South Beach'),
 Row(area='Lakeshore'),
 Row(area='Bayview Hunters Point'),
 Row(area='Visitacion Valley'),
 Row(area='Inner Richmond'),
 Row(area='Nob Hill'),
 Row(area='Oceanview/Merced/Ingleside'),
 Row(area='Outer Richmond'),
 Row(area='Treasure Island'),
 Row(area='Chinatown'),
 Row(area='Mission'),
 Row(area='Excelsior'),
 Row(area='Twin Peaks'),
 Row(area='Seacliff'),
 Row(area='Sunset/Parkside'),
 Row(area='Presidio Heights'),
 Row(area='Portola'),
 Row(area='Golden Gate Park'),
 Row(area='Glen Park'),
 Row(area='McLaren Park'),
 Row(area='Presidio'),
 Row(area='Tenderloin'),
 Row(area='Potrero Hill'),
 Row(area='Outer Mission'),
 Row(area='Castro/Upper Mark

In [45]:
# how many calls are there per area?
spark.sql("""
SELECT area, COUNT(*) AS count
FROM calls
GROUP BY area
ORDER BY count DESC
""").toPandas()

,area,count
0,Tenderloin,809041
1,South of Market,583631
2,Mission,543289
3,Financial District/South Beach,402331
4,Bayview Hunters Point,326423
5,Sunset/Parkside,233135
6,Western Addition,222759
7,Nob Hill,198957
8,Outer Richmond,159714
9,Hayes Valley,148946


In [47]:
# how many calls are per group/type
spark.sql("""
SELECT Call_Type_Group, Call_type, COUNT(*) AS count
FROM calls
GROUP BY Call_Type_Group, Call_type
ORDER BY count DESC
LIMIT 5
""").toPandas()

,Call_Type_Group,Call_type,count
0,None,Medical Incident,1783402
1,Potentially Life-Threatening,Medical Incident,1448636
2,Non Life-threatening,Medical Incident,716653
3,None,Structure Fire,416250
4,Alarm,Alarms,366116


In [49]:
# for each group, what percentage of calls are represented by the biggest type?
spark.sql("""
SELECT Call_Type_Group, MAX(count)/SUM(count) as perc
FROM (
    SELECT Call_Type_Group, Call_type, COUNT(*) AS count
    FROM calls
    GROUP BY Call_Type_Group, Call_type
)
GROUP BY Call_Type_Group
""").toPandas()

,Call_Type_Group,perc
0,Alarm,0.486623
1,None,0.632953
2,Potentially Life-Threatening,0.942302
3,Non Life-threatening,0.924175
4,Fire,0.383007


In [56]:
from pyspark.sql.functions import expr, col

In [63]:
# same question, but with Spark DataFrame API
(spark.table("calls")
 .groupby("Call_Type_Group", "Call_type")
 .count()
 .groupby("Call_Type_Group")
 .agg(expr("MAX(count) / SUM(count)").alias("perc"))                # aggregate
).toPandas()

,Call_Type_Group,perc
0,Alarm,0.486623
1,None,0.632953
2,Potentially Life-Threatening,0.942302
3,Non Life-threatening,0.924175
4,Fire,0.383007


In [70]:
# what are the 3 SMALLEST call number for each area?
spark.sql("""
SELECT area, Call_Number, row_number() OVER (PARTITION BY area ORDER BY Call_Number ASC) AS rownum
FROM calls
""").where("rownum <= 3").toPandas()

,area,Call_Number,rownum
0,Bayview Hunters Point,1030119,1
1,Bayview Hunters Point,1040031,2
2,Bayview Hunters Point,1040078,3
3,Bernal Heights,1040089,1
4,Bernal Heights,1040245,2
...,...,...,...
121,West of Twin Peaks,1040020,2
122,West of Twin Peaks,1040115,3
123,Western Addition,1030128,1
124,Western Addition,1030128,2
